Подсчет ROI в разрезе типов источников трафика. Т.е. значения ROI по типам. Типы определяются следующим образом:

    - если source равен 'google' или 'yandex', то проверяем medium:

        - для medium 'seo' или 'sem' тип источника равен 'search engines seo'

        - для medium 'brand' - тип источника равен 'search engines brand'

        - для остальных случаев тип источника равен 'search engines undefined'

    - если условие не выполнено, то тип источника равен 'other'

In [1]:
"""создадим функцию для определения типа источника. 
передадим ей в качестве аргумента все данные по заказу (т. е. всю строку line)"""

def source_type_classification( line ):
    """
    Пример строки на входе функции:
    ['40443', '05.10.2016 23:18', '1010', '0,000925926', 'seo', 'google', '0', '6243', '20,20']
    Функция классифицирует источник трафика в зависимости от значения source и medium.
    Возвращает название типа источника по правилам:
        - если source равен 'google' или 'yandex', то проверяем medium:
            - для medium 'seo' или 'sem' возвращаем 'search engines seo'
            - для medium 'brand' - возвращаем 'search engines brand'
            - для остальных случаев возвращаем 'search engines undefined'
        - если условие не выполнено, то возвращаем 'other'
    """
    source = line[5]
    medium = line[4]
    
    if source == 'google' or source == 'yandex':
        if medium == 'seo' or medium == 'sem':
            return 'search engines seo'
        elif medium == 'brand':
            return 'search engines brand'
        else:
            # если вдруг встретится другой вариант, то ставим "undefined"
            return 'search engines undefined'
    else:
        return 'other'
line=['40177', '05.10.2016 3:23', '1041', '0,0033796', 'email', 'promo', '0', '6128', '13,20']

source_type_classification(line)

'other'

In [ ]:
""" создадим словарь. в нем указаны все типы и их количество """ 
## на самом деле это здесь не нужно
source_type_stats={}
with open('data_no_header.txt','r') as f:
    for line in f:
        line=line.strip().split('\t')
        source=line[5]
        medium=line[4]
        sourceType=source_type_classification(line)
        if sourceType in source_type_stats:
            source_type_stats[sourceType]+=1
        else:
            source_type_stats[ sourceType ] = 1

source_type_stats.items()

In [2]:
## параграф 2  ---  РАБОТАЕТ ВЕРНО!!!!
""" сформируем новые словари""" 
cpa_commission = {'burgerclub': 0.3,'food-delivery': 0.25}
cpc_commission = {'city-magazine': 7,'foody': 9}
fixed_commission = 4
def costs_classification( amount_paid, source ): ### ВОЗРАЩАЕТ КОМИССИЮ 
    
    """
    Функция по названию канала source возвращает размер комиссии.
    Размеры комиссий берутся из словарей cpa_commission, cpc_commission и значения fixed_commission
    """
# если источник source входит в словарь cpa_commission, то возвращаем долю от выручки
    if source in cpa_commission:
        return amount_paid * cpa_commission[source]
# в случае партнеров CPC стоимость расходов постоянная
    if source in cpc_commission:
        return cpc_commission[ source ]
# если ни один случай не подходит, то возвращаем фиксированную стоимость fixed_commission
    return fixed_commission
costs_classification( 10.0, 'foody' )  ### ДАДИМ ФУНКЦИИ ПРИМЕР ИЗ ЭКСЕЛЯ

9

In [3]:
## параграф 3 - вроде все верно но надо проверить строку 10
##### СЧИТАЕМ ROI 
def expense_and_income( line ):
    """
    Функция для строки line возвращает итоговый расход и доход
    """
    source = line[5]
    MediumType = source_type_classification(line) #возвращает функция из параграфа 1. д.б. верно
    amount_paid = float( line[-1].replace( ',', '.' ) ) #берем из строки файла
    cost = float( line[6].replace( ',', '.' ) ) #берем из строки файла
    partner_comission = costs_classification( amount_paid, source ) # ФУНЦИЯ из параграфа 2 НАДО ПРОВЕРИТЬ!!!
    return MediumType, cost+partner_comission, amount_paid


expense_and_income(['40443', '05.10.2016 23:18', '1010', '0,000925926', 'seo', 'yandex', '1', '6243', '20,20'])



('search engines seo', 5.0, 20.2)

In [4]:
## параграф 4 ---  РАБОТАЕТ ВЕРНО!!!! но надо проверить строку 8
# Теперь прогоним через функцию весь файл и запишем результат в словарь roi_stats

roi_stats = {}
with open( 'data_no_header.txt', 'r' ) as f:
    for line in f:
        line = line.strip().split('\t')
        MediumType, cost, income = expense_and_income( line ) ### из параграфа 3 НАДО ПРОВЕРИТЬ!!!
        if MediumType in roi_stats:
            roi_stats[ MediumType ][ 'cost' ] += cost
            roi_stats[ MediumType ][ 'income' ] += income
        else:
            roi_stats[ MediumType ] = {}
            roi_stats[ MediumType ][ 'cost' ] = cost
            roi_stats[ MediumType ][ 'income' ] = income
            
roi_stats
# search engines seo': {'cost': 686.2699999999992, 'income': 2902.4} - сходится с EXCEL
# search engines brand': {'cost': 52.0, 'income': 234.60000000000005} - сходится с EXCEL
# other : в EXCELL  1487,00 и 340 - НЕ сходится !!!

{'search engines seo': {'cost': 686.2699999999992, 'income': 2902.4},
 'other': {'cost': 541.1059999999999, 'income': 1992.7999999999997},
 'search engines brand': {'cost': 52.0, 'income': 234.60000000000005}}

In [5]:
## параграф 5 ---  РАБОТАЕТ ВЕРНО!!!!
for MediumType, data in roi_stats.items(): #dict.items() - возвращает пары (ключ, значение).
    data['roi'] = ( data['income'] - data['cost'] ) / data['cost']

for MediumType, data in roi_stats.items(): #dict.items() - возвращает пары (ключ, значение).
    print( '{}\t{:.2f}'.format( MediumType, data['roi'] ) )

search engines seo	3.23
other	2.68
search engines brand	3.51
